In [7]:
import pandas as pd
import numpy as np

In [8]:
def msd_straight_forward(r):
    shifts = np.arange(len(r))
    msds = np.zeros(shifts.size)    

    for i, shift in enumerate(shifts):
        diffs = r[:-shift if shift else None] - r[shift:]
        sqdist = np.square(diffs).sum(axis=1)
        msds[i] = sqdist.mean()

    return msds


def autocorrFFT(x):
  N=len(x)
  F = np.fft.fft(x, n=2*N)  #2*N because of zero-padding
  PSD = F * F.conjugate()
  res = np.fft.ifft(PSD)
  res= (res[:N]).real   #now we have the autocorrelation in convention B
  n=N*np.ones(N)-np.arange(0,N) #divide res(m) by (N-m)
  return res/n #this is the autocorrelation in convention A


def msd_fft(r):
  N=len(r)
  D=np.square(r).sum(axis=1) 
  D=np.append(D,0) 
  S2=sum([autocorrFFT(r[:, i]) for i in range(r.shape[1])])
  Q=2*D.sum()
  S1=np.zeros(N)
  for m in range(N):
      Q=Q-D[m-1]-D[N-m]
      S1[m]=Q/(N-m)
  return S1-2*S2

def msd_for_df(df):
    #print(df)
    if 'z' in df.columns and all(np.isfinite(df.z.values)):
        r = np.asarray([df.x.values, df.y.values, df.z.values]).T
    else:
        r = np.asarray([df.x.values, df.y.values]).T
    msd = msd_fft(r)
    
    return msd

In [0]:
# all_df['msd'] = exp_df.groupby(['date', 'condition', 'experiment', 'cell', 'particle'],).apply(msd_for_df)

In [17]:
df = pd.DataFrame([[12, 'c1_001.oif', 9.0, 'double_trial', 0, 9.0, 1, 0.0, 1, 50.5, 16.0, 1.0, 'cover_2_dsam', '20180505']
, [87, 'c1_001.oif', -1.0, 'double_trial', 0, -1.0, 2, 0.034482758620689655, 2, 9.0, 41.793103448275865, 1.0, 'cover_2_dsam', '20180505']
, [23, 'c1_001.oif', 1.0, 'double_trial', 0, 1.0, 3, 0.0, 3, 49.0, 42.0, 1.0, 'cover_2_dsam', '20180505']
, [94, 'c1_001.oif', -1.0, 'double_trial', 1, -1.0, 1, 0.031914893617021274, 2, 8.617021276595745, 41.840425531914896, 1.0, 'cover_2_dsam', '20180505']
, [20, 'c1_001.oif', 1.0, 'double_trial', 1, 1.0, 2, 0.0, 3, 49.0, 42.2, 1.0, 'cover_2_dsam', '20180505']
, [12, 'c1_001.oif', 6.0, 'double_trial', 2, 6.0, 1, 0.0, 4, 50.5, 16.0, 1.0, 'cover_2_dsam', '20180505']
, [80, 'c1_001.oif', 1.0, 'double_trial', 2, 1.0, 2, 0.0, 2, 8.8875, 42.4125, 1.0, 'cover_2_dsam', '20180505']
, [93, 'c1_001.oif', -2.0, 'double_trial', 3, -2.0, 1, 0.06451612903225806, 2, 9.21505376344086, 41.774193548387096, 1.0, 'cover_2_dsam', '20180505']
, [12, 'c1_001.oif', -2.0, 'double_trial', 3, -2.0, 2, 0.8333333333333334, 5, 50.5, 42.0, 1.0, 'cover_2_dsam', '20180505']
, [12, 'c1_001.oif', -1.0, 'double_trial', 4, -1.0, 2, 0.08333333333333333, 5, 51.0, 42.5, 1.0, 'cover_2_dsam', '20180505']], 
 columns=['area', 'cell', 'distance', 'experiment', 'frame', 'full_erode',
       'label', 'mean_intensity', 'particle', 'x', 'y', 'z', 'condition',
       'date'])

In [16]:
def msd_for_augusto(df):
    #print(df)
    if 'z' in df.columns and all(np.isfinite(df.z.values)):
        r = np.asarray([df.x.values, df.y.values, df.z.values]).T
    else:
        r = np.asarray([df.x.values, df.y.values]).T
    msd = msd_fft(r)
    df['msd'] = msd
    return df

In [48]:
msd_for_augusto(df)

,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,9.094947e-13
1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,1.341414e+03
2,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,1.319555e+03
3,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,1.247603e+03
4,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,8.984265e+02
5,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,6.683177e+02
6,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,1.057067e+03
7,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,1.365100e+03
8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,1.220250e+03
9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,7.025000e+02


In [73]:
augusto_df=df.groupby(['date', 'condition', 'experiment', 'cell', 'particle']).apply(msd_for_augusto).sort_values(by=['particle','x']).reset_index()
augusto_df

,index,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
1,3,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,3.546889e-01
2,6,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,3.791689e-01
3,1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,4.547474e-13
4,7,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,4.660571e-02
5,2,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
6,4,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,4.000000e-02
7,5,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
8,8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
9,9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,5.000000e-01


In [58]:
resultado = df.groupby(('date', 'condition', 'experiment', 'cell', 'particle'), as_index=False).apply(msd_for_augusto)
resultado

,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,4.547474e-13
2,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
3,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,3.546889e-01
4,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,4.000000e-02
5,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
6,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,3.791689e-01
7,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,4.660571e-02
8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,5.000000e-01


In [60]:
resultado.sort_values(by=['particle','x'])

,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
3,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,3.546889e-01
6,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,3.791689e-01
1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,4.547474e-13
7,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,4.660571e-02
2,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
4,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,4.000000e-02
5,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,5.000000e-01


In [76]:
all_df = pd.DataFrame()
groupers = ['date', 'condition', 'experiment', 'cell', 'particle']
for this_attrs, this_df in df.groupby(groupers):
    this_df['msd'] = msd_for_df(this_df)
    all_df = all_df.append(this_df, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
all_df.sort_values(by=['particle','x'],inplace=True)
all_df = all_df.reset_index()

In [83]:
all_df == augusto_df

,index,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
7,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [84]:
all_df

,index,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
1,2,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,3.546889e-01
2,3,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,3.791689e-01
3,1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,4.547474e-13
4,4,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,4.660571e-02
5,5,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
6,6,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,4.000000e-02
7,7,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
8,8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
9,9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,5.000000e-01


In [85]:
augusto_df

,index,area,cell,distance,experiment,frame,full_erode,label,mean_intensity,particle,x,y,z,condition,date,msd
0,0,12,c1_001.oif,9.0,double_trial,0,9.0,1,0.000000,1,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
1,3,94,c1_001.oif,-1.0,double_trial,1,-1.0,1,0.031915,2,8.617021,41.840426,1.0,cover_2_dsam,20180505,3.546889e-01
2,6,80,c1_001.oif,1.0,double_trial,2,1.0,2,0.000000,2,8.887500,42.412500,1.0,cover_2_dsam,20180505,3.791689e-01
3,1,87,c1_001.oif,-1.0,double_trial,0,-1.0,2,0.034483,2,9.000000,41.793103,1.0,cover_2_dsam,20180505,4.547474e-13
4,7,93,c1_001.oif,-2.0,double_trial,3,-2.0,1,0.064516,2,9.215054,41.774194,1.0,cover_2_dsam,20180505,4.660571e-02
5,2,23,c1_001.oif,1.0,double_trial,0,1.0,3,0.000000,3,49.000000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
6,4,20,c1_001.oif,1.0,double_trial,1,1.0,2,0.000000,3,49.000000,42.200000,1.0,cover_2_dsam,20180505,4.000000e-02
7,5,12,c1_001.oif,6.0,double_trial,2,6.0,1,0.000000,4,50.500000,16.000000,1.0,cover_2_dsam,20180505,0.000000e+00
8,8,12,c1_001.oif,-2.0,double_trial,3,-2.0,2,0.833333,5,50.500000,42.000000,1.0,cover_2_dsam,20180505,0.000000e+00
9,9,12,c1_001.oif,-1.0,double_trial,4,-1.0,2,0.083333,5,51.000000,42.500000,1.0,cover_2_dsam,20180505,5.000000e-01


In [19]:
pd.__version__

'0.22.0'